In [ ]:
import requests
from bs4 import BeautifulSoup
import Geant4
from Geant4 import *
import g4py.NISTmaterials
import g4py.ezgeom
from g4py.ezgeom import G4EzVolume
import g4py.EMSTDpl
import g4py.ParticleGun
import g4py.MedicalBeam
import os
import os.path
import base64  
from IPython.display import HTML
import ipywidgets as widgets
from ipywidgets import Button, Layout
style = {'description_width': 'initial'}
from ipywidgets import IntSlider
import math

In [ ]:
particle_type_selector = widgets.ToggleButtons(options={'Photons':'gamma', 'Positrons':'e+', 'Electrons':'e-'}, 
                                               description='Particle type:'.decode('utf-8'), value='gamma')
display(particle_type_selector)

number_of_particles_slider = widgets.IntSlider(
    value=20,
    min=1,
    max=100,
    step=9,
    description='Number of particles:',
    continuous_update=False,
    readout=True,
    readout_format='d', style=style
)
display(number_of_particles_slider)

phantom_material_dropdown = widgets.Dropdown(
    options={'Water':'G4_WATER', 'Air':'G4_AIR', 'Iron': 'G4_Fe', 'Lead':'G4_Pb', 'Copper': 'G4_Cu', 'Gold': 'G4_Au', 'Sodium iodide': 'G4_SODIUM_IODIDE', 'Aluminum': 'G4_Al'},
    value='G4_WATER',
    description='Phantom material:', layout=Layout(width='100%'), style=style
)
display(phantom_material_dropdown)

world_material_selector = widgets.ToggleButtons(style=style, options={'Air':'G4_AIR', 'Empty space'.decode('utf-8'):'G4_Galactic', 'Water':'G4_WATER'}, description='World material:', value='G4_AIR')
display(world_material_selector)

phantom_range_slider = widgets.IntRangeSlider(
    value=[100, 120],
    min=0,
    max=200,
    step=10,
    description='Phantom location and width:'.decode('utf-8'),
    readout=True,
    readout_format='d', layout=Layout(width='100%', height='80px'), style=style
)
display(phantom_range_slider)

particle_energy_slider = widgets.FloatSlider(
    value=.1,
    min=0.01,
    max=.2,
    step=.01,
    description='Particle energy (in MeV):',
    continuous_update=False,
    readout=True,
    readout_format='.2f', style=style, layout=Layout(width='80%', height='80px')
)
display(particle_energy_slider)

In [ ]:
phantom_zwidth = math.floor(phantom_range_slider.value[1] - phantom_range_slider.value[0])*cm
rand_engine = Ranlux64Engine()
HepRandom.setTheEngine(rand_engine)
HepRandom.setTheSeed(20050830L)
g4py.NISTmaterials.Construct()
g4py.ezgeom.Construct()
g4py.EMSTDpl.Construct()
g4py.ParticleGun.Construct()
world_material = G4Material.GetMaterial(world_material_selector.value)
g4py.ezgeom.SetWorldMaterial(world_material)
g4py.ezgeom.ResizeWorld(120.*cm, 120.*cm, 400.*cm)
phantom_material = G4Material.GetMaterial(phantom_material_dropdown.value)
phantom = G4EzVolume("PhantomBox")
phantom_zlocation = math.floor(phantom_range_slider.value[0])*cm
phantom.CreateBoxVolume(phantom_material, 100.0 * cm, 100.0 * cm, phantom_zwidth)
phantom.SetColor(0., 0.9, 1.0)
phantom.PlaceIt(G4ThreeVector(0.*cm, 0.*cm, phantom_zlocation))
beam = g4py.MedicalBeam.Construct()
beam.particle = particle_type_selector.value
beam.kineticE = particle_energy_slider.value*MeV
beam.sourcePosition = G4ThreeVector(0.*cm, 0.*cm, -90.*cm)
beam.fieldXY = [120.*cm, 120.*cm]
beam.SSD = 190.*cm
gApplyUICommand("/run/initialize")
gApplyUICommand("/vis/viewer/flush")
gApplyUICommand("/vis/open VRML2FILE")
gApplyUICommand("/vis/viewer/refresh")
gApplyUICommand("/vis/scene/create")
gApplyUICommand("/vis/scene/add/volume")
gApplyUICommand("/vis/drawVolume")
gApplyUICommand("/vis/modeling/trajectories/create/drawByParticleID")
gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set gamma white")
gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set proton blue")
gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set e- yellow")
gApplyUICommand("/vis/modeling/trajectories/drawByParticleID-0/set e+ green")
gApplyUICommand("/vis/sceneHandler/attach")
gApplyUICommand("/vis/viewer/set/style wireframe")
gApplyUICommand("/vis/viewer/set/viewpointThetaPhi 70. 10.")
gApplyUICommand("/vis/viewer/zoom 1.")
gApplyUICommand("/tracking/storeTrajectory 1")
gApplyUICommand("/vis/scene/add/trajectories")
gApplyUICommand("/vis/scene/add/hits")
gApplyUICommand("/vis/scene/add/trajectories smooth")
gApplyUICommand("/vis/scene/endOfEventAction accumulate")
gRunManager.Initialize()
gRunManager.BeamOn(number_of_particles_slider.value)
os.rename("g4_00.wrl", "simulacion.wrl")

In [3]:
import base64  
from IPython.display import HTML
with open('simulacion.wrl', 'r') as file:
    data = file.read()
b64 = base64.b64encode(data.encode())
payload = b64.decode()
html = '<a download="simulacion.wrl" href="data:text/wrl;base64,{payload}" target="_blank">{title}</a>'
html = html.format(payload=payload, title='Download raw WRL file',filename='simulacion.wrl')
HTML(html)

KeyError: 'simulacion'

In [ ]:
API_ENDPOINT = "https://doc.instantreality.org/tools/x3d_encoding_converter/convert/"

with open('simulacion.wrl', 'r') as file:
    wrl_data = file.read()

data = {'input_encoding': 'CLASSIC',
        'output_encoding': 'HTML5', 'input_code': wrl_data}

r = requests.post(url=API_ENDPOINT, data=data)
soup = BeautifulSoup(r.text)
html_file = open("simulacion.html", "w")
html_file.write(soup.find('div', {'class': 'source'}).text)
html_file.close()

In [ ]:
from IPython.display import IFrame
IFrame(src='./simulacion.html', width=700, height=600)

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")